In [1]:
import pandas as pd 
from glob import glob
import os
import sys
import tkinter as tk
from tkinter import filedialog
import csv
import json
import openpyxl
import xlsxwriter

In [ ]:
# For Folder
# root=tk.Tk()
# root.withdraw()
# folder1=filedialog.askdirectory(title="Browse the folder...")

In [ ]:
# For Folder
# files_df1=sorted(glob(folder1+'/*.csv'))
# df=pd.concat((pd.read_csv(file).assign(filename = file) for file in files_df1),ignore_index=True)

In [2]:
# For File
root=tk.Tk()
root.withdraw()
file1=filedialog.askopenfilename(title="Browse the required file...")

In [3]:
# For File
df=pd.read_csv(file1)

In [4]:
df['event_data'].astype('str')
df['event_data']=df['event_data'].apply(json.loads)
df2=pd.json_normalize(df['event_data'])
df3=pd.concat([df.drop(['event_data'],axis=1),df2],axis=1)
del df,df2
df3["type"]=df3["type"].replace("apron_management.","",regex=True)
cols = []
count = 1
for column in df3.columns:
    if column == 'time':
        cols.append(f'time.{count}')
        count+=1
        continue
    cols.append(column)
df3.columns = cols
df3.drop('version', inplace=True, axis=1)

In [ ]:
###STAGE_CHANGES###
# Filters <- Process != alarm_housekeeper AND type = dgs_state_changes AND event_type = afedockStateChangeEvent
stage_changes=df3[(df3['process'] != "alarm_housekeeper") & (df3['type'] == "dgs_state_changes")& (df3['event_type'] == "SafedockStateChangeEvent")]
stage_changes['VDGS_state']=stage_changes.prev_safedock_state.str.cat(stage_changes.safedock_state,sep='_To_')
stage_changes['Flight_ID']=stage_changes['carrier'].str.cat(stage_changes['flight_nr'],sep=' ')
stage_cols=["id", "type", "time.2", "ifplid", "ac_type", "apron_id", "stand_id", "VDGS_state","event_classification", "Flight_ID"]
stage_changes=stage_changes[stage_cols]
stage_changes.rename(columns = {"id":"ID", "type":"Type", "time.2":"Time", "ifplid":"IFPLID", "ac_type":"Aircraft_Type", "apron_id":"Apron_ID", "stand_id":"Stand_ID", "VDGS_state":"VDGS_State","event_classification":"Event_Classification"},inplace=True)
stage_changes['Type']=stage_changes['Type'].replace(to_replace ="dgs_state_changes",value ="Stage Changes")

In [10]:
check=df3[(df3['type'] == "alarms")& (df3['event_type'] == "AlarmOccurrencesEvent")]
check=check['alarm.alarm-description'].value_counts().reset_index()

In [ ]:
list=df3['alarm.alarm-description'].unique()
list_df=pd.DataFrame(list)

In [ ]:
###ALARMS###
filter_col = [col for col in df3 if col.startswith('alarm.')]
alarms=df3[(df3['type'] == "alarms")&(df3['event_src'] == "Safedock")& (df3['event_type'] == "AlarmOccurrencesEvent")]
alarm_cols=["id", "host", "process", "time.1", "type", "site", "time.2", "event_classification",'apron_id', "alarm", "event_src", "event_type", "location_info", "schedule_info"]
alarm_cols=alarm_cols+filter_col
alarms=alarms[alarm_cols]
alarms=alarms.explode('schedule_info')
alarms=alarms.explode('location_info')
alarm_error_list = ['ApronBlocked','AutoCalibrationError','BridgeNotIn','ConfigurationError - SystemIntegrity','GateBlocked','IDFailed - EngineCheckFailed','IDFailed - IdLostTrackCloseToStop','IDFailed - NoseHeightFailed','IDFailed - ProfileFailed','IDFailed - VerAnalysisFailedTooClose','Major display error - faulty LED board detected','Minor display error - faulty LEDs detected','SBUStop - AzimuthStop','SBUStop - LdmDataRate','SBUStop - ScanFailed','ViewBlocked']
alarms=alarms[(alarms['alarm.alarm-description'].isin(alarm_error_list) )]
alarm_location=pd.json_normalize(alarms['location_info'])
alarm_location=alarm_location.add_prefix('location_info.')
alarm_schedule=pd.json_normalize(alarms['schedule_info'])
alarm_schedule=alarm_schedule.add_prefix('schedule_info.')
alarms=pd.concat([alarms.reset_index(),alarm_location.reset_index(),alarm_schedule.reset_index()],axis=1)
alarms['Flight_ID']=alarms['schedule_info.carrier'].str.cat(alarms['schedule_info.flight_nr'],sep=' ')
alarms=alarms[['id','type', 'site', 'time.2','event_classification','alarm.alarm-id','location_info.apron_id','alarm.stand-id','alarm.alarm-code', 'alarm.alarm-description','schedule_info.ac_type','Flight_ID']]
alarms.rename(columns = {'id':'ID','type':'Type','time.2':'Time','alarm.alarm-id':'Alarm_ID','location_info.apron_id':'Apron_ID','alarm.stand-id':'Stand_ID','alarm.alarm-code':'Alarm_Code', 'alarm.alarm-description':'Alarm_Description','schedule_info.ac_type':'Aircraft_Type'}, inplace = True)

In [11]:
###TOTAL_ALARMS###
filter_col = [col for col in df3 if col.startswith('alarm.')]
total_alarms=df3[(df3['type'] == "alarms")& (df3['event_type'] == "AlarmOccurrencesEvent")]
total_alarm_cols=["id", "host", "process", "time.1", "type", "site", "time.2", "event_classification",'apron_id', "alarm", "event_src", "event_type", "location_info", "schedule_info"]
total_alarm_cols=total_alarm_cols+filter_col
total_alarms=total_alarms[total_alarm_cols]
total_alarms=total_alarms.explode('schedule_info')
total_alarms=total_alarms.explode('location_info')
total_alarm_error_list = ['ApronBlocked','AutoCalibrationError','BridgeNotIn','ConfigurationError - SystemIntegrity','GateBlocked','IDFailed - EngineCheckFailed','IDFailed - IdLostTrackCloseToStop','IDFailed - NoseHeightFailed','IDFailed - ProfileFailed','IDFailed - VerAnalysisFailedTooClose','Interface Status Alarm: AGL NOK','Interface Status Alarm: CCTV NOK','Interface Status Alarm: NTP NOK','Interface Status Alarm: SMR Degraded','Interface Status Alarm: SMR NOK','Interface Status Alarm: SRV1 NOK','Interface Status Alarm: SRV2 NOK','Interface Status Alarm: UFIS NOK','Major display error - faulty LED board detected','Minor display error - faulty LEDs detected','Safedock Connection Lost','SBUStop - AzimuthStop','SBUStop - LdmDataRate','SBUStop - ScanFailed','ViewBlocked','Walktest required for All','Walktest required for C:T;C:1;C:2;C:3','Walktest required for C:T;C:1;C:2;C:3;C:4;C:5','Walktest required for C:T;C:2;C:3;C:4;C:5','Walktest required for L:T;L:1;L:2;L:3','Walktest required for R:T;R:1;R:2']
total_alarms=total_alarms[(total_alarms['alarm.alarm-description'].isin(total_alarm_error_list) )]
total_alarm_location=pd.json_normalize(total_alarms['location_info'])
total_alarm_location=total_alarm_location.add_prefix('location_info.')
total_alarm_schedule=pd.json_normalize(total_alarms['schedule_info'])
total_alarm_schedule=total_alarm_schedule.add_prefix('schedule_info.')
total_alarms=pd.concat([total_alarms.reset_index(),total_alarm_location.reset_index(),total_alarm_schedule.reset_index()],axis=1)
total_alarms['Flight_ID']=total_alarms['schedule_info.carrier'].str.cat(total_alarms['schedule_info.flight_nr'],sep=' ')
total_alarms=total_alarms[['id','type', 'site', 'time.2','event_classification','alarm.alarm-id','location_info.apron_id','alarm.stand-id','alarm.alarm-code', 'alarm.alarm-description','schedule_info.ac_type','Flight_ID']]
total_alarms.rename(columns = {'id':'ID','type':'Type','time.2':'Time','alarm.alarm-id':'Alarm_ID','location_info.apron_id':'Apron_ID','alarm.stand-id':'Stand_ID','alarm.alarm-code':'Alarm_Code', 'alarm.alarm-description':'Alarm_Description','schedule_info.ac_type':'Aircraft_Type'}, inplace = True)

In [ ]:
###PIVOT HELPER####

error_mapping={'Minor display error - faulty LEDs detected':'Technical','IDFailed - IdLostTrackCloseToStop':'Operational','ApronBlocked':'Operational','IDFailed - NoseHeightFailed':'Operational','GateBlocked':'Operational','Major display error - faulty LED board detected':'Technical','IDFailed - VerAnalysisFailedTooClose':'Operational','AutoCalibrationError':'Technical','BridgeNotIn':'Operational','SBUStop - AzimuthStop':'Technical','SBUStop - ScanFailed':'Technical','IDFailed - EngineCheckFailed':'Operational','ConfigurationError - SystemIntegrity':'Technical','SBUStop - LdmDataRate':'Technical','ViewBlocked':'Operational','IDFailed - ProfileFailed':'Operational'}
stage_change_alarms=pd.concat([stage_changes,alarms],axis=0,ignore_index=True)
stage_change_alarms['Error_Type'] = stage_change_alarms['Alarm_Description'].map(error_mapping)
stage_change_alarms['Time']=pd.to_datetime(stage_change_alarms['Time'])

In [ ]:
###sosFileSize###
sosFileSize=df3[(df3['type'] == "safedock_file_downloads")]
sos_columns=['id', 'time.1',  'stand_id','file_name','file_size','file_destination_path']
sosFileSize=sosFileSize[sos_columns]

In [ ]:
###aibt_aobt###
aibt_aobt=df3[(df3['type'] == "block_on_off")]
aibt_aobt['carrier_nr']=aibt_aobt['carrier'].str.cat(aibt_aobt['flight_nr'],sep=' ')
aibt_aobt_cols=["id", "type", "stand_id", "ac_type", "carrier_nr", "detected_aibt", "detected_aobt", "trigger_source"]
aibt_aobt=aibt_aobt[aibt_aobt_cols]

In [ ]:
###Commands###
commands=df3[((df3['type'] == "block_on_off")|(df3['type'] == "closed_for_maintenance")|(df3['type'] == "dgs_commands")|(df3['type'] == "dgs_state_changes"))&((df3['event_type'] != "AircraftScanEvent")&(df3['event_src'] != "SAM"))]
commands['carrier_nr']=commands['carrier'].str.cat(commands['flight_nr'],sep=' ')
commands=commands[~(commands['event_type'].isin(['EmergencyStopEvent','MaintenanceEvent','SafedockStateChangeEvent']))] 
commands=commands[(commands['event_type'].isin(['AbortDockingEvent','ParkOffEvent','ParkOnEvent','ShutdownScheduleEvent','StartDockingEvent']))] 
commands_columns=['id','time.2','event_type','stand_id','event_src','carrier_nr','ac_type']
commands=commands[commands_columns]
commands['time.2']=pd.to_datetime(commands['time.2'])
commands.rename(columns = {'time.2':'time'},inplace=True)


In [ ]:
###SAM_sequencing###
sam_sequencing=df3[(df3['type'] == "flight_updates")&(df3['flight_plan.fields.sequencing-state']=='Completed')&(df3['event_type']!='FlightTerminatedEvent')]
sam_sequencing['carrier_nr']=sam_sequencing['carrier'].str.cat(sam_sequencing['flight_nr'],sep=' ')
sam_sequencing['SAM_sequencing_state']=sam_sequencing['flight_plan_diff.fields.sequencing-state.from'].str.cat(sam_sequencing['flight_plan_diff.fields.sequencing-state.to'],sep='_To_')
# sam_filter_col = [col for col in sam_sequencing if col.startswith('flight_plan')]
sam_col=['id','flight_plan.timestamps.SEQT','flight_plan.fields.stand','flight_plan.fields.flight-plan-type','carrier_nr','flight_plan.fields.update-counter', 'ac_type','flight_plan.fields.actual-dgs-aircraft-type','flight_plan.fields.tow-in','SAM_sequencing_state','flight_plan.timestamps.EIBT','flight_plan.timestamps.DETECTED_AIBT','flight_plan.timestamps.EOBT','flight_plan.timestamps.DETECTED_AOBT']
# sam_col=sam_col+sam_filter_col
sam_sequencing=sam_sequencing[sam_col]
sam_sequencing['flight_plan.timestamps.SEQT']=pd.to_datetime(sam_sequencing['flight_plan.timestamps.SEQT'])
sam_sequencing.rename(columns={'flight_plan.timestamps.SEQT':'SEQT','flight_plan.fields.stand':'Stand','flight_plan.fields.flight-plan-type':'flight-plan-type','flight_plan.fields.update-counter':'update-counter','flight_plan.fields.actual-dgs-aircraft-type':'actual-dgs-aircraft-type','flight_plan.fields.tow-in':'tow-in','flight_plan.timestamps.EIBT':'EIBT','flight_plan.timestamps.DETECTED_AIBT':'DETECTED_AIBT','flight_plan.timestamps.EOBT':'EOBT','flight_plan.timestamps.DETECTED_AOBT':'DETECTED_AOBT'},inplace=True)

In [ ]:
###Interface###
interface=df3[(df3['type'] == "equipment_state_changes") & df3['event_type'].isin(['GPUOffEvent','GPUOnEvent','PLBDockedEvent','PLBInEvent'])]
interface_col=["id","time.2","event_type","equipment_id"]
interface=interface[interface_col]
interface['time.2']=pd.to_datetime(interface['time.2'])
interface.rename(columns={"time.2":"time"},inplace=True)

In [ ]:
###Downgraded###
downgraded=df3[(df3['type'] == "dgs_downgrade")]
downgraded['carrier_nr']=downgraded['carrier'].str.cat(downgraded['flight_nr'],sep=' ')
downgraded_cols=["id", "time.2", "ifplid", "carrier_nr", "reg_nr", "ac_type", "apron_id", "stand_id", "event_type"]
downgraded=downgraded[downgraded_cols]

In [ ]:
###RIDS###
rids=df3[(df3['process'] == "stand_engine")&(df3['type'] == "rids")]

In [ ]:
with pd.ExcelWriter("C:/Users/jamit\Desktop/filename.xlsx") as writer:

    # use to_excel function and specify the sheet_name and index
    # to store the dataframe in specified sheet
    stage_change_alarms.to_excel(writer, sheet_name="VDGS_stagechanges_alarms", index=False)
    stage_changes.to_excel(writer, sheet_name="VDGS_stagechanges", index=False)
    alarms.to_excel(writer, sheet_name="VDGS_alarms", index=False)
    sosFileSize.to_excel(writer, sheet_name="VDGS_sosFileSizw", index=False)
    aibt_aobt.to_excel(writer, sheet_name="VDGS_aibt_aobt", index=False)
    commands.to_excel(writer, sheet_name="VDGS_commands", index=False)
    sam_sequencing.to_excel(
        writer, sheet_name="VDGS_sam_sequencing", index=False)
    interface.to_excel(writer, sheet_name="VDGS_interface", index=False)

In [9]:
with pd.ExcelWriter("C:/Users/jamit\Desktop/list_error.xlsx") as writer:
  check.to_excel(writer, sheet_name="error_list", index=False)
  total_alarms.to_excel(writer, sheet_name="All_Errors", index=False)